In [1]:
import os
import numpy as np
import pywt
import pandas as pd
import pickle as pkl
from matplotlib import pyplot as plt

In [2]:
# 필요 변수
DATA_POINTS_PER_FILE = 2560
TIME_PER_REC = 0.1
SAMPLING_FREQ = 25600 # 25.6 KHz
SAMPLING_PERIOD = 1.0/SAMPLING_FREQ

WIN_SIZE = 20
WAVELET_TYPE = 'morl'

VAL_SPLIT = 0.1

SEQ_LEN = 5 # sequence length

In [3]:
def load_df(pkz_file):  #load data frame from pickle file
    with open(pkz_file, 'rb') as f:
        df=pkl.load(f)
    return df

In [4]:
def df_row_ind_to_data_range(ind):
    return (DATA_POINTS_PER_FILE*ind, DATA_POINTS_PER_FILE*(ind+1))

In [5]:
def extract_feature_image(ind, feature_name='horiz accel'):
    data_range = df_row_ind_to_data_range(ind)
    data = df[feature_name].values[data_range[0]:data_range[1]]

    data = np.array([np.mean(data[i:i+WIN_SIZE]) for i in range(0, DATA_POINTS_PER_FILE, WIN_SIZE)])

    coef, _ = pywt.cwt(data, np.linspace(1,128,128), WAVELET_TYPE)

    coef = np.log2(coef**2+0.001)

    coef = (coef - coef.min())/(coef.max() - coef.min())
    return coef

In [12]:
main_dir = 'D:/data/'

In [19]:
pkz_file = main_dir + 'noisy_04_bearing1_2.pkz'
df = load_df(pkz_file)
df.head()

,hour,minute,second,microsecond,horiz accel,vert accel
0,8,47,5,196910.0,0.690192,-0.158381
1,8,47,5,196950.0,0.091153,-0.104491
2,8,47,5,196990.0,-0.123156,0.802257
3,8,47,5,197030.0,0.211616,-0.688070
4,8,47,5,197070.0,-0.005533,-0.410238


In [20]:
no_of_rows = df.shape[0]
no_of_files = int(no_of_rows / DATA_POINTS_PER_FILE)
print(no_of_rows, no_of_files)

2229760 871


In [21]:
data = {'x': [], 'y': []}
for i in range(0, no_of_files):
    coef_h = extract_feature_image(i, feature_name='horiz accel')
    coef_v = extract_feature_image(i, feature_name='vert accel')
    x_ = np.array([coef_h, coef_v])
    y_ = i/(no_of_files-1)
    data['x'].append(x_)
    data['y'].append(y_)
data['x']=np.array(data['x'])
data['y']=np.array(data['y'])

assert data['x'].shape==(no_of_files, 2, 128, 128)
print(no_of_files, data['x'].shape, data['y'].shape)

871 (871, 2, 128, 128) (871,)


In [22]:
out_file = main_dir+'noisy_04_all_bearing1_2.pkz'
with open(out_file, 'wb') as f:
    pkl.dump(data, f)